In [1]:
import tensorflow as tf

A TensorFlow variable is the recommended way to represent shared, persistent state your program manipulates. This guide covers how to create, update, and manage instances of tf.Variable in TensorFlow.

Variables are created and tracked via the tf.Variable class. A tf.Variable represents a tensor whose value can be changed by running ops on it. Specific ops allow you to read and modify the values of this tensor. Higher level libraries like tf.keras use tf.Variable to store model parameters.

In [2]:
tensor = tf.constant(5)
variable = tf.Variable([[2,3],[4,5]])

bool_variable = tf.Variable([False , True , False  , True])
complex_variable = tf.Variable([5+4j , 6+5j])

In [4]:
print("Shape:", variable.shape)
print("Dtype:",variable.dtype)
print("Converted in Numpy:",variable.numpy)

Shape: (2, 2)
Dtype: <dtype: 'int32'>
Converted in Numpy: <bound method BaseResourceVariable.numpy of <tf.Variable 'Variable:0' shape=(2, 2) dtype=int32, numpy=
array([[2, 3],
       [4, 5]], dtype=int32)>>



A variable looks and acts like a tensor, and, in fact, is a data structure backed by a tf.Tensor. Like tensors, they have a dtype and a shape, and can be exported to NumPy.

Most tensor operations work on variables as expected, although variables cannot be reshaped.

In [7]:
print("Variable:",variable)
print("\n Viewed as a Tensor:", tf.convert_to_tensor(variable))
print("\n Index of Highest Value:", tf.argmax(variable))

# It creates a new Tensor . and not reshaped the original tensor
print("\n Copying and Reshaping:", tf.reshape(variable, ([1,4])))

Variable: <tf.Variable 'Variable:0' shape=(2, 2) dtype=int32, numpy=
array([[2, 3],
       [4, 5]], dtype=int32)>

 Viewed as a Tensor: tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)

 Index of Highest Value: tf.Tensor([1 1], shape=(2,), dtype=int64)

 Copying and Reshaping: tf.Tensor([[2 3 4 5]], shape=(1, 4), dtype=int32)



As noted above, variables are backed by tensors. You can reassign the tensor using tf.Variable.assign. Calling assign does not (usually) allocate a new tensor; instead, the existing tensor's memory is reused.

In [9]:
a = tf.Variable([2,3])

a.assign([4,5])

try:
  a.assign([1,2,7])
except Exception as e: print(e)
print("Variable:",a)

Shapes (2,) and (3,) are incompatible
Variable: <tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([4, 5], dtype=int32)>



If you use a variable like a tensor in operations, you will usually operate on the backing tensor.

Creating new variables from existing variables duplicates the backing tensors. Two variables will not share the same memory.

In [10]:
a = tf.Variable([4,5])

b = tf.Variable(a)
a.assign([2,3])

print(a.numpy())
print(b.numpy())
print(a.assign_add([2,3]).numpy()) #[4,6]
print(a.assign_sub([4,6]).numpy()) #[0,0]

[2 3]
[4 5]
[4 6]
[0 0]


## LifeCycle, Naming and Watching

In Python-based TensorFlow, tf.Variable instance have the same lifecycle as other Python objects. When there are no references to a variable it is automatically deallocated.

Variables can also be named which can help you track and debug them. You can give two variables the same name.

In [11]:
a = tf.Variable(tensor , name="Aakash")

b = tf.Variable(tensor + 1 , name="Aakash")
# These are elementwise-unequal, despite having the same name
print(a==b)

tf.Tensor(False, shape=(), dtype=bool)


Variable names are preserved when saving and loading models. By default, variables in models will acquire unique variable names automatically, so you don't need to assign them yourself unless you want to.

Although variables are important for differentiation, some variables will not need to be differentiated. You can turn off gradients for a variable by setting trainable to false at creation. An example of a variable that would not need gradients is a training step counter.

In [12]:
step_counter = tf.Variable(1 , trainable=False)

## Placing Variable and Tensors

For better performance, TensorFlow will attempt to place tensors and variables on the fastest device compatible with its dtype. This means most variables are placed on a GPU if one is available.

However, we can override this. In this snippet, we can place a float tensor and a variable on the CPU, even if a GPU is available. By turning on device placement logging, we can see where the variable is placed.

Note: Although manual placement works, using can be a more convenient and scalable way to optimize your computation.

If you run this notebook on different backends with and without a GPU you will see different logging. Note that logging device placement must be turned on at the start of the session

In [14]:
with tf.device('CPU:0'):

  # Create some tensors
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
  c = tf.matmul(a, b)

print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


It's possible to set the location of a variable or tensor on one device and do the computation on another device. This will introduce delay, as data needs to be copied between the devices.

You might do this, however, if you had multiple GPU workers but only want one copy of the variables.

In [15]:

with tf.device('CPU:0'):
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.Variable([[1.0, 2.0, 3.0]])

with tf.device('GPU:0'):
  # Element-wise multiply
  k = a * b

print(k)

tf.Tensor(
[[ 1.  4.  9.]
 [ 4. 10. 18.]], shape=(2, 3), dtype=float32)


## Low Level Tensorflow Code

In this notebook, we will start by reviewing the main operations on Tensors in TensorFlow and understand how to manipulate TensorFlow Variables. We explain how these are compatible with python built-in list and numpy arrays.

In [16]:
import numpy as np
import matplotlib.pyplot as plt

In [17]:
x = tf.constant([4,5,6])
x

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([4, 5, 6], dtype=int32)>

In [18]:
x = tf.Variable(2.0 , name = "my_variable")
x

<tf.Variable 'my_variable:0' shape=() dtype=float32, numpy=2.0>

In [20]:
x.assign(45.2)
x

<tf.Variable 'my_variable:0' shape=() dtype=float32, numpy=45.2>

In [22]:
x.assign_add(4.0)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=49.2>

In [23]:
x.assign_sub(2)
x

<tf.Variable 'my_variable:0' shape=() dtype=float32, numpy=47.2>

## Point Wise Operations

In [24]:
a = tf.Variable([1,2,3])
b = tf.Variable([1,1,1])

c = tf.add(a,b)
d = a + b

print("C:",c)
print("D:",d)

C: tf.Tensor([2 3 4], shape=(3,), dtype=int32)
D: tf.Tensor([2 3 4], shape=(3,), dtype=int32)


In [25]:
a = tf.Variable([1,2,3])
b = tf.Variable([1,1,1])

c = tf.multiply(a,b)
d = a * b

print("C:",c)
print("D:",d)

C: tf.Tensor([1 2 3], shape=(3,), dtype=int32)
D: tf.Tensor([1 2 3], shape=(3,), dtype=int32)


In [27]:
a = tf.Variable([3.,4.,5.])
b = tf.math.exp(a)

print("B:",b)

B: tf.Tensor([ 20.085537  54.59815  148.41316 ], shape=(3,), dtype=float32)


## Python Interoperability


In [29]:
# python List
a_py = [1,2]
b_py = [3,4]

In [30]:
tf.add(a_py , b_py)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([4, 6], dtype=int32)>

In [31]:
# Numpy Arrays
a_np = np.array([1,2,3])
b_np = np.array([2,3,4])

In [32]:
tf.add(a_np , b_np)

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([3, 5, 7])>

In [34]:
# Tensorflow list via tf.constant
a_tf = tf.constant([1,2])
b_tf = tf.constant([2,3])

In [35]:
tf.add(a_tf , b_tf)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([3, 5], dtype=int32)>

In [36]:
a_tf.numpy()

array([1, 2], dtype=int32)

**End for Now**
****